# Creación del modelo final:

En el siguiente Notebook se prepara el script final donde se integran ambos modelos. Posteriormente, este Notebook se adaptará para convertirse en un archivo '.py', permitiendo su uso como un endpoint en el entorno de producción que se montará haciendo uso de EC2 de AWS.

En este Notebook se muestra todo el código y explicaciones importantes de este, pero la explicación detallada se encuentra en la memoria del proyecto.

## Preparación del entorno:

### Instalación de dependencias:

In [1]:
!pip install --upgrade ultralytics
!pip install opencv-python-headless

   ---------------------------------------- 0.0/911.6 kB ? eta -:--:--
   --------------------------------------- 911.6/911.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.61
    Uninstalling ultralytics-8.3.61:
      Successfully uninstalled ultralytics-8.3.61


### Importación de librerías:

In [2]:
import os
import cv2
import numpy as np

from ultralytics import YOLO

### Definición de funciones:

Funcion que guarda los frames de un video en un array.

In [3]:
def convertVideoToFramesArray(video_path):
	frames_array = []
	cap = cv2.VideoCapture(video_path)

	if not cap.isOpened():
			print("Error al abrir el archivo de video")
	else:
			total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

			for frame_index in range(total_frames):
					ret, frame = cap.read()
					if not ret:
							print(f"No se pudo leer el frame {frame_index}")
							continue

					frames_array.append(frame)

			cap.release()
	return frames_array

Función que permite crear txt con las coordenadas del cuadrado de la etiqueta

In [4]:
def createTxtFile(damagesArray, fileName, coordenadas):
    damages_dict = {
        "broken lamp": 0,
        "glass shatter": 1,
        "scratch": 2,
        "dent": 3,
        "tire flat": 4,
    }

    if os.path.exists(fileName):
        os.remove(fileName)

    content = ''

    if 'no-damage' not in damagesArray:
        for element in damagesArray:
            damageNum = damages_dict[element]
            content = content + str(damageNum) + ' ' + coordenadas + '\n'

    with open(fileName, "w") as file:
        file.write(content)


Funcion que calcula la similitud entre dos imágenes mediante la correlación de sus histogramas en escala de grises.

In [5]:
def calcular_similitud_histograma(imagen1, imagen2):
    # Convertir las imágenes a escala de grises
    imagen1_gray = cv2.cvtColor(imagen1, cv2.COLOR_BGR2GRAY)
    imagen2_gray = cv2.cvtColor(imagen2, cv2.COLOR_BGR2GRAY)
    
    # Calcular los histogramas de las dos imágenes
    hist1 = cv2.calcHist([imagen1_gray], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([imagen2_gray], [0], None, [256], [0, 256])
    
    # Normalizar los histogramas
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()

    # Calcular la correlación entre los histogramas
    similitud = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    
    # Convertir la similitud en porcentaje
    similitud_porcentaje = similitud
    
    return similitud_porcentaje

Funcion que recorta una región específica de una imagen original utilizando coordenadas normalizadas y devuelve la imagen recortada.

In [6]:
def obtener_imagen(parte_de_la_imagen_xyxyn, imagen_original):
    if len(parte_de_la_imagen_xyxyn) != 4 or any(not isinstance(coord, (float, int)) for coord in parte_de_la_imagen_xyxyn):
        raise ValueError("Las coordenadas deben ser una lista de cuatro valores numéricos [x_min, y_min, x_max, y_max].")
    
    x_min, y_min, x_max, y_max = parte_de_la_imagen_xyxyn
    h, w, _ = imagen_original.shape
    
    x_min = max(0, int(x_min * w))
    y_min = max(0, int(y_min * h))
    x_max = min(w, int(x_max * w))
    y_max = min(h, int(y_max * h))
    
    imagen_recortada = imagen_original[y_min:y_max, x_min:x_max]
    
    if imagen_recortada.size == 0:
        print(f"Recorte vacío: coordenadas fuera de los límites de la imagen.")
    
    return imagen_recortada

Función que recibe un daño y una parte del vehículo y devuelve True si se puede dar dicho daño en dicha parte (por ejemplo, rallajo en puerta trasera) y False si no se puede dar dicho daño en dicha parte (por ejemplo, faro roto en puerta trasera).

In [7]:
def is_damage_possible(nombre_de_clase_2, numero_de_clase):
    diccionario_combinaciones_posibles = {
        0: ['scratch', 'dent'],
        1: ['tire flat'],
        2: ['glass shatter'],
        3: ['scratch', 'dent'],
        4: ['scratch', 'dent'],
        5: ['scratch', 'dent'],
        6: [],
        7: ['glass shatter'],
        8: ['glass shatter'],
        9: ['scratch', 'dent'],
        10: ['broken lamp'],
        11: ['tire flat'],
        12: ['glass shatter'],
        13: ['scratch', 'dent'],
        14: ['scratch', 'car_damages_confdent'],
        15: ['broken lamp'],
        16: [],
        17: ['scratch', 'dent'],
        18: ['scratch', 'dent'],
        19: ['scratch', 'dent'],
        20: ['scratch', 'dent'],
    }
    if nombre_de_clase_2 in diccionario_combinaciones_posibles[numero_de_clase]:
        return True
    else:
        return False

Función que devuelve True si la imagen 'imagen_aux' se parece al menos en un 70% a alguna de las imagenes del array 'imagenes' y False en caso contrario.

In [8]:
def misma_imagen(imagenes, imagen_aux):
    for element in imagenes:
        parecido = calcular_similitud_histograma(element, imagen_aux)
        if parecido >= 0.8:
            return True
    return False

Función que guarda una imagen en una ruta determinada. Tambien almacena dicha imagen en formato array al array 'imagenes'.

In [9]:
def guardar_imagen(ruta_para_guardar, imagen_aux):
    # Guardo la imagen:
    cv2.imwrite(ruta_para_guardar, imagen_aux)

Función que añade al array 'damages' la información de las predicciones. Este array se enviará desde el endpoint al cliente que hace la llamada al endpoint.

In [10]:
def anadir_info_del_dano(ruta_para_guardar, conf_damage, nombre_de_clase_1, nombre_de_clase_2, damages):
    damages_traduction = {
	"broken lamp": "faro roto",
		"glass shatter": "cristal roto",
		"scratch": "rallada",
		"dent": "bolladura",
		"tire flat": "rueda pinchada"
}
    
    info = {
        'car_damage_route' : ruta_para_guardar,
        'car_damage_conf' : conf_damage,
        'car_part_name' : nombre_de_clase_1,
        'car_damage' : damages_traduction[nombre_de_clase_2],
    }

    damages.append(info)
    return damages

Función que devuelve True si la confianza de la predicción está por encima del valor mínimo para cada etiqueta; devuelve False en caso contrario.

In [11]:
def is_confidence_sufficient(nombre_de_clase_2, conf_damage):

    mult = .8
    
    car_damages_config = {
            "broken lamp": 0.94 * mult,
            "glass shatter": 0.88 * mult,
            "scratch": 0.88 * mult,
            "dent": 0.81 * mult,
            "tire flat": 0.95 * mult
        }

    if conf_damage >= car_damages_config[nombre_de_clase_2]:
        return True
    else:
        return False

Función que, dado un array de una imagen y las coordenadas de un cuadrado, devuelve la misma imagen con el cuadrado marcado en color rojo.

In [12]:
def draw_red_square(image_array, coords):
    image_with_square = image_array.copy()

    height, width, _ = image_with_square.shape

    x_min = int(coords[0] * width)
    y_min = int(coords[1] * height)
    x_max = int(coords[2] * width)
    y_max = int(coords[3] * height)

    color = (0, 0, 255)
    thickness = 2

    cv2.rectangle(image_with_square, (x_min, y_min), (x_max, y_max), color, thickness)

    return image_with_square

Se definen las variables de la ruta donde están los modelos:

In [13]:
car_damgs_model_path = os.getcwd() + '/recursos adicionales/car_damages/train/weights/best.pt'
car_parts_model_path = os.getcwd() + '/recursos adicionales/car_parts/train/weights/best.pt'

Se define la variable de la ruta de un video:

In [18]:
video_path = os.getcwd() + '/as.mp4'

Se define la función principal que devuelve las predicciones (en la memoria escrita se detalla como funciona cada trozo de código):

In [19]:
def main(car_parts_model_path, car_damgs_model_path, video_path):
    num_of_image = 0
    damages = []
    imagenes = []
    frames_array = []
    damages_arr = []
    
    # Paso los videos a un array de frames:
    frames_array = convertVideoToFramesArray(video_path)

    # Itero sobre todos los frames del video:
    for frame_index, frame in enumerate(frames_array):
        # Predigo las partes del vehículo:
        results = YOLO(car_parts_model_path).predict(source=frame, save=False, show=False, device='cpu', conf=0.8, verbose=False)
        
        for element in results[0].boxes:
            parte_de_la_imagen = obtener_imagen(element.xyxyn.tolist()[0], frame)
            numero_de_clase = int(element.cls.tolist()[0])
            nombre_de_clase_1 = results[0].names[numero_de_clase]
    
            # Predigo los daños del vehículo:
            results_2 = YOLO(car_damgs_model_path).predict(source=parte_de_la_imagen, save=False, show=False, device='cpu', verbose=False)
    
            # Itero sobre los resultados de las predicciones de los daños del vehículo detectados:
            for element_2 in results_2[0].boxes:
                numero_de_clase_2 = int(element_2.cls.tolist()[0])
                parte_de_la_imagen_2 = obtener_imagen(element_2.xyxyn.tolist()[0], parte_de_la_imagen)
                nombre_de_clase_2 = results_2[0].names[numero_de_clase_2]
                conf_damage = float(element_2.conf.tolist()[0])
            
                if is_damage_possible(nombre_de_clase_2, numero_de_clase) and is_confidence_sufficient(nombre_de_clase_2, conf_damage):
                        damages_arr.append([frame_index, numero_de_clase, numero_de_clase_2])
                        num_of_image = num_of_image + 1
                        imagen_aux = cv2.resize(parte_de_la_imagen, (256, 256))
                        imagen_aux_square = draw_red_square(imagen_aux, element_2.xyxyn.tolist()[0])
                    
                        if len(imagenes) == 0 or not misma_imagen(imagenes, imagen_aux_square):
                            print('entro')
                            # Guardo la imagen en la carpeta del dataset:
                            os.makedirs('retrained-dataset/images/', exist_ok=True)
                            ruta_imagen_ds = 'retrained-dataset/images/'  + video_path.split('/')[-1].split('.')[0] + '_' + str(num_of_image) + '.jpg'
                            guardar_imagen(ruta_imagen_ds, imagen_aux)

                            # Guardo la label en la carpeta del dataset:
                            os.makedirs('retrained-dataset/labels/', exist_ok=True)
                            ruta_label_ds = 'retrained-dataset/labels/'  + video_path.split('/')[-1].split('.')[0] + '_' + str(num_of_image) + '.txt'
                            createTxtFile([nombre_de_clase_2], ruta_label_ds, ' , '.join(map(str, element_2.xyxyn.tolist()[0])))

                            # Guardo la imagen para poder verla desde la web:
                            os.makedirs('web-images/', exist_ok=True)
                            ruta_imagen_web = 'web-images/' + video_path.split('/')[-1].split('.')[0] + '_' + str(num_of_image) + '.jpg'
                            guardar_imagen(ruta_imagen_web, imagen_aux_square)

                            # Guardo la imagen en el array:
                            imagenes.append(imagen_aux_square)
            
                            damages = anadir_info_del_dano(ruta_imagen_web, conf_damage, nombre_de_clase_1, nombre_de_clase_2, damages)
    return damages

In [20]:
a = main(car_parts_model_path, car_damgs_model_path, video_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Usuario\\Documents\\MASTER\\recursos adicionales\\Notebooks\\recursos adicionales\\car_parts\\train\\weights\\best.pt'